"Datasæt: https://www.opendata.dk/city-of-copenhagen/monumenter
CSV: https://wfs-kbhkort.kk.dk/k101/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=k101:monumenter&outputFormat=csv&SRSNAME=EPSG:4326

1. Hvor mange monumenter er der i København?

2. Hvor mange monumenter bliver vedligeholdt? Dvs. graffitirenhold = ja

3. Lav en funktion som kan finde koordinaterne på et monument baseret på monumentets id eller navn?
	F.eks:
		def monumentById(monumentId):
			return coordinates
	
3.a Vis monument som bliver returneret i metoden, på et kort over københavn ved brug af plotting. (Se afsnittet om 'Folium and Bokeh' under notebooks/03-3 Plotting)

4. Find navnet på monumentet med x og y koordinaterne eller længde- og breddegraderne?
	
	x = 724407.424966
	y = 6175719.798486
	
	MULTIPOINT ((12.555485308174104 55.69383926601615))

5. Lav en metode der optegner alle monumenterne på kortet ved brug af plotting.

6. Gør kortet interaktiv så navnet på monumenterne vises når man trykker på et plot. (Se afsnittet om 'Interactive plots with bokeh' under notebooks/03-3 Plotting)"

In [21]:
import pandas as pd
df = pd.read_csv('monumenter.csv')
df.head()

,FID,id,navn,loebenummer,web_id,x,y,longitude,latitude,graffitirenhold,wkb_geometry
0,monumenter.1,49690.0,Efterår,115.0,167.0,724712.263534,6.179443e+06,12.577001,55.708568,Ja,MULTIPOINT ((12.577000674229012 55.70856778229...
1,monumenter.2,49865.0,H.W. Meyer,41.0,93.0,723888.874104,6.178082e+06,12.562810,55.696741,Ja,MULTIPOINT ((12.562810238950165 55.69674138769...
2,monumenter.3,49866.0,Sigurd Fafnersbane og slange,314.0,353.0,724293.902081,6.178228e+06,12.569361,55.697869,Ja,MULTIPOINT ((12.569361482261764 55.69786887177...
3,monumenter.4,50058.0,Satyr med bacchusbarn,21.0,67.0,724122.061749,6.176324e+06,12.565077,55.680871,Ja,MULTIPOINT ((12.56507667919525 55.680870776003...
4,monumenter.5,50063.0,Talerstol,290.0,330.0,724157.041343,6.178457e+06,12.567375,55.699982,Ja,MULTIPOINT ((12.567375194374895 55.69998215768...


In [31]:
import csv

filename = 'monumenter.csv'
with open(filename) as f:
    reader = csv.reader(f)
    header_row = next(reader)

    for index, column_header in enumerate(header_row): 
        print(index, column_header)

df.info()

0 FID
1 id
2 navn
3 loebenummer
4 web_id
5 x
6 y
7 longitude
8 latitude
9 graffitirenhold
10 wkb_geometry
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 389 entries, 0 to 388
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   FID              389 non-null    object 
 1   id               386 non-null    float64
 2   navn             369 non-null    object 
 3   loebenummer      369 non-null    float64
 4   web_id           366 non-null    float64
 5   x                389 non-null    float64
 6   y                389 non-null    float64
 7   longitude        389 non-null    float64
 8   latitude         389 non-null    float64
 9   graffitirenhold  363 non-null    object 
 10  wkb_geometry     389 non-null    object 
dtypes: float64(7), object(4)
memory usage: 33.6+ KB


In [20]:
import csv

filename = 'monumenter.csv'
ids = set([])

with open(filename) as f:
    reader = csv.reader(f)
    header_row = next(reader)

    for row in reader:
        ids.add(row[0])
        
print("Der er " + str(len(ids)) + " monumenter i KBH")

Der er 389 monumenter i KBH


In [30]:
print("Der er", len(df), "monumenter i KBH")
print("Der er", len(df[df['navn'].notnull()]), "monumenter i KBH")

Der er 389 monumenter i KBH
Der er 369 monumenter i KBH
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 389 entries, 0 to 388
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   FID              389 non-null    object 
 1   id               386 non-null    float64
 2   navn             369 non-null    object 
 3   loebenummer      369 non-null    float64
 4   web_id           366 non-null    float64
 5   x                389 non-null    float64
 6   y                389 non-null    float64
 7   longitude        389 non-null    float64
 8   latitude         389 non-null    float64
 9   graffitirenhold  363 non-null    object 
 10  wkb_geometry     389 non-null    object 
dtypes: float64(7), object(4)
memory usage: 33.6+ KB


In [4]:
import csv

filename = 'monumenter.csv'
grafitirenhold = set([])

with open(filename) as f:
    reader = csv.reader(f)
    header_row = next(reader)
     
    for row in reader:
        if row[9] == 'Ja':
            grafitirenhold.add(row[0])
        
print("Der er " + str(len(grafitirenhold)) + " monumenter i KBH, som bliver grafiti vedligeholdt")

Der er 255 monumenter i KBH, som bliver grafiti vedligeholdt


In [39]:
import csv
filename = 'monumenter.csv'

monument_nr = input("indtast FID på det monument du ønsker: ")

def get_monument(args):
    data =[]
    with open(filename) as f:
        reader = csv.reader(f)
        header_row = next(reader)
    
        for row in reader:
            if row[0] == args:
                data.append(row[8])
                data.append(row[7])
                data.append(row[2])
                return data 
        
        
monument = get_monument(monument_nr)
print(monument[0])

indtast FID på det monument du ønsker: monumenter.1
55.708568


In [83]:
import json
import requests
import folium
import urllib


def get_city_location(city='Copenhagen'):
    """Get the location coordinates from OpenStreetMaps"""
    url_nomatim_api = 'https://nominatim.openstreetmap.org/search'
    r = requests.get(url_nomatim_api, params={'format': 'json', 'city': city})
    results = r.json()  # Potentially many matches
    #print('results: ',results)
    location = results[0]
    lat, lon = float(location['lat']), float(location['lon'])
    return lat, lon

map_osm = folium.Map(location=(get_city_location()), zoom_start=10) # show the map
# add marker one by one on the map

folium.Marker(
    location=[monument[0], monument[1]],
    popup=monument[2],
).add_to(map_osm)

# Show the map again
#map_osm.save('./osm2.html')
map_osm



In [85]:
import json
import requests
import folium
import urllib


def get_city_location(city='Copenhagen'):
    """Get the location coordinates from OpenStreetMaps"""
    url_nomatim_api = 'https://nominatim.openstreetmap.org/search'
    r = requests.get(url_nomatim_api, params={'format': 'json', 'city': city})
    results = r.json()  # Potentially many matches
    #print('results: ',results)
    location = results[0]
    lat, lon = float(location['lat']), float(location['lon'])
    return lat, lon

data = df[['navn','longitude', 'latitude']]
data = data[data['navn'].notnull()] 
print(data)

map_osm = folium.Map(location=(get_city_location()), zoom_start=10) # show the map
# add marker one by one on the map
for i in range(0,len(data)):
    folium.Marker(
      location=[data.iloc[i]['latitude'], data.iloc[i]['longitude']],
      popup=data.iloc[i]['navn'],
   ).add_to(map_osm)

# Show the map again
#map_osm.save('./osm2.html')
map_osm



                             navn  longitude   latitude
0                         Efterår  12.577001  55.708568
1                      H.W. Meyer  12.562810  55.696741
2    Sigurd Fafnersbane og slange  12.569361  55.697869
3           Satyr med bacchusbarn  12.565077  55.680871
4                       Talerstol  12.567375  55.699982
..                            ...        ...        ...
382           Adam Oehlenschläger  12.586189  55.679726
383                Ludvig Holberg  12.585880  55.679668
384               Siddende kvinde  12.587330  55.670313
386   Modstandsbevægelsens faldne  12.595237  55.668805
387             Kongens Livjægere  12.588918  55.697472

[369 rows x 3 columns]


In [86]:
#df[df['navn'].isin(['H.C. Andersen'])]
try:
    varX = df[df['x'].isin(['724407.424966'])].navn.item()
    varY = df[df['y'].isin(['6175719.798486'])].navn.item()
    if varX == varY:
        print(varX)
except:
    print("kordinat ikke fundet")



H.C. Andersen


In [72]:
df[df['wkb_geometry'].isin(['MULTIPOINT ((12.555485308174104 55.69383926601615))'])].navn.item()

'Dansende piger'